In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
from utils import *

In [ ]:
import random
import json

In [ ]:
all_data = []
with open('perturbed1.txt', 'r') as f:
    all_data = f.readlines()
print(len(all_data))
with open('wrong.txt', 'r') as f:
    all_data += f.readlines()
print(len(all_data))
random.shuffle(all_data)

In [ ]:
json.loads(all_data[3])

In [ ]:
checker = MistralChecker(model_path='mistralai/Mistral-7B-Instruct-v0.2', device='cuda')

In [ ]:
data_test = json.loads(all_data[3])
data_test

In [ ]:
checker.raw_checker(question=data_test['question'], correct_ans=data_test['correct_answer'], model_ans=data_test['perturbed_answer'])

In [ ]:
correct = 0
total = 0
diff = 0
errors = []
for i, data in enumerate(all_data):
    try:
        if i % 100 == 1:
            print(correct/(total+1e-3), correct, total)
        data = json.loads(data)
        messages = [
            {"role": "user", "content": "context: You are a helpful abiding assistant. Your inputs will consist of a question and a correct answer, and a answer from a model. Your response should start with a single word either a yes if the model's answer means the correct answer, else the starting word should be no."},
            {"role": "assistant", "content": "Okay, I will do that."},
        ]
        expected_checker_ans = 1
        if 'perturbed_answer' in data:
            ans = checker.raw_checker(question=data['question'], correct_ans=data['correct_answer'], model_ans=data['perturbed_answer'])
            ans = int(ans[0])
        else:
            ans = checker.raw_checker(question=data['question'], correct_ans=data['correct_answer'], model_ans=data['wrong_answer'])
            expected_checker_ans = 0
            ans = int(ans[0])
        
        # print(decoded, expected_checker_ans)
        if ans == expected_checker_ans:
            correct += 1
        total += 1
    except Exception as e:
        print(correct/total, correct, total)
        error = e
        errors.append(data)
        continue
correct/total, correct, total, errors